# Purpose

This notebook will use the kaggle dataset "Titanic - Machine Learning from Disaster".  
to demonstrate logistic regression and a pipeline to do a lot of dataset manipulation.  

The goal isn't necessarily to win the Titanic competion - which isn't possible anyway because there are perfect classifiers out there, but to demonstrate some techniques and beat a naive baseline.

In [26]:
from typing import Tuple

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder

Gather the data and a little preprocessing.  
*  create my own train/test split
*  drop some columns that don't make good 'out of the box' features

In [2]:
train_competition = pd.read_csv('./data/train.csv')
test_competition = pd.read_csv('./data/test.csv')

In [3]:
print(f'Train: {train_competition.shape}')
print(f"Test: {test_competition.shape}")

Train: (891, 12)
Test: (418, 11)


In [4]:
train_competition.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [5]:
# Drop a few columns that wouldn't make good 'out of the box' features
train_x = train_competition.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin"], axis=1)
train_y = train_competition.loc[:,["Survived"]]

In [6]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.1, stratify=train_y)

In [7]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(801, 7)
(801, 1)
(90, 7)
(90, 1)


In [24]:
train_x.dtypes

Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

# Baseline

One of the most important tasks with ML and supervised learning is to establish a baseline.

A naive baseline doesn't use any input features, for a binary classification problem such as this it is akin to predicting the majority class in all cases.

Without a naive baseline you cannot be certain a ML model is able to learn from input features.

In [18]:
# The majority class is dead
baseline = (train_y.shape[0] - train_y.sum())/train_y.shape[0]

In [23]:
print(f'Baseline: {round(baseline.iloc[0], 2)}')

Baseline: 0.62


# Pipeline

Take the approach as treating all training data categorical.

Pclass, Sex, SibSp, Parch, Embarked will be one-hot encoded  
Age and Fare will be bucketed and one-hot encoded

In [25]:
# convert non-categorical columns to categorical prior to encoding
train_x["Pclass"] = train_x["Pclass"].astype('category')
train_x["Sex"] = train_x["Sex"].astype('category')
train_x["SibSp"] = train_x["SibSp"].astype('category')
train_x["Parch"] = train_x["Parch"].astype('category')
train_x["Embarked"] = train_x["Embarked"].astype('category')

In [ ]:
model_pipeline = Pipeline([
    ('impute_nan', pass),
    ('categorical_encoder',
     OneHotEncoder(drop="if_binary", 
                   handle_unknown="infrequent_if_exist", 
                   sparse_output=False)),
    ('numeric_binner', 
     KBinsDiscretizer(encode='onehot-dense'))

])